In [ ]:
#default_exp training.tuner

# Tuning API
> Modules for task fine-tuning

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.all import *
from fastai.metrics import *
from fastai.losses import *
from fastai.optimizer import *
# NOTE: Placeholder imports, remove once we are ready for release

from transformers import default_data_collator

from adaptnlp.training.data import *

## Integration with [fastai](https://docs.fast.ai)

Since `fastai` is a _very_ lightweight framework that is easily approachable and incorporates state-of-the-art ideas, `AdaptNLP` bridges the gap between HuggingFace and fastai, allowing you to train with their framework through the `*Tuner` classes

In [ ]:
#export
class _AdaptiveLearner(Learner):
    """
    A base fastai `Learner` that overrides `_split` and `_do_one_batch` to
    have it work with HuggingFace datasets and models
    """
    def _split(self, b):
        "Assign `self.xb` to model input and labels"
        self.xb = b
        if 'labels' in b.keys(): self.yb = b['labels'].unsqueeze(0)
    
    def _do_one_batch(self):
        "Move a batch of data to a device, get predictions, calculate the loss, and perform backward pass"
        self.xb = {k:v.to(self.device) for k,v in self.xb.items()} # See if `to_device` fixes this
        self.yb = self.yb.to(self.device)
        out = self.model(**self.xb)
        self.pred = out['logits'].to(self.device)
        self('after_pred')
        self.loss_grad = out['loss'].to(self.device)
        self.loss = self.loss_grad.clone()
        self('after_loss')
        if not self.training or not len(self.yb): return
        self('before_backward')
        self.loss_grad.backward()
        self._with_events(self.opt.step, 'step', CancelStepException)
        self.opt.zero_grad()

In [ ]:
#export
_a = {'OneCycle':'fit_one_cycle', 'CosineAnnealing':'fit_flat_cos', 'SGDR':'fit_sgdr'}
mk_class('Strategy', **_a, doc_string='Class for fitting strategies with typo-proofing')

In [ ]:
#export
class AdaptiveTuner:
    """
    A base `Tuner` that interfaces with `AdaptiveLearner` with specific exposed functions
    """
    @delegates(_AdaptiveLearner.__init__)
    def __init__(self, expose_fastai:bool=False, **kwargs):
        self._tuner = _AdaptiveLearner(**kwargs)

        exposed_attrs = ['dls', 'model', 'loss_func', 'metrics']
        for attr in exposed_attrs:
            setattr(self, attr, getattr(self._tuner, attr))
        if expose_fastai:
            cls = self.__class__
            self.__class__ = cls.__class__("AdaptiveTuner", (cls, _AdaptiveLearner), kwargs)
            
    def tune(
        self,
        epochs:int, # Number of epochs to train for
        lr:float = None, # If None, finds a new LR and uses suggestion_method
        strategy:Strategy = Strategy.OneCycle,
        callbacks = [], # Extra fastai Callbacks
        **kwargs ## kwargs for the fit function
        
    ):
        "Fine tune `self.model` for `epochs` with an `lr` and `strategy`"
        func = getattr(self, strategy, getattr(self._tuner, strategy, None))
        for attr in 'epochs,lr,cbs'.split(): 
            if attr in kwargs.keys(): kwargs.pop(attr)
        func(epochs, lr, cbs=callbacks, **kwargs)
        
    @delegates(Learner.lr_find)
    def lr_find(self, **kwargs): return self._tuner.lr_find(**kwargs)
    
    def save(self, file:Union[Path,str], with_opt=True, pickle_protocol=2): 
        file = join_path_file(kwargs['file'], self.path/self.model_dir, ext='.pth')
        if rank_distrib(): return # Don't save if child proc
        opt = getattr(self, 'opt', None)
        if opt is None: with_opt = False
        state = get_model(self.model).state_dict()
        state = {'model':state}
        if with_opt: state['opt'] = opt.state_dict()
        state['model_name_or_path'] = self.model.name_or_path
        torch.save(state, file, pickle_protocol=pickle_protocol)
        return file
    
    def load(self, file:Union[Path,str], device=None, with_opt=True, strict=True):
        if device is None and hasattr(self.dls, 'device'): device = self.dls.device
        if self.opt is None: self.create_opt()
        file = join_path_file(file, self.path/self.model_dir, ext='.pth')
        distrib_barrier()
        if isinstance(device, int): device = torch.device('cuda', device)
        elif device is None: device='cpu'
        state = torch.load(file, map_location=device)
        hasopt = 'opt' in state.keys()
        model_state = state['model']
        get_model(self.model).load_state_dict(model_state, strict=strict)
        if hasopt and with_opt:
            try: self.opt.load_state_dict(state['opt'])
            except:
                if with_opt: warn("Could not load the optimizer state.")
        elif with_opt: warn("Saved file doesn't contain an optimizer state")
        return self
    
for attr in ['lr_find', 'save', 'load']: 
    setattr(getattr(AdaptiveTuner, attr), '__doc__', getattr(_AdaptiveLearner, attr).__doc__)

The constructor of the `AdaptiveTuner` class has an optional `expose_fastai_api` parameter. When set to `True`, the `Tuner` inherits fastai's `Learner`, so every attribute of the `Learner` is available to you. This is only recommended for those very familiar with the fastai API.

Otherwise, you have access to six* functions in each class:
  - `tune`
  - `lr_find`
  - `predict` (Coming soon)
  - `save`
  - `load`
  - `export` (Coming soon)

In [ ]:
#export
from transformers import AutoModelForSequenceClassification

In [ ]:
#export
class SequenceClassificationTuner(AdaptiveTuner):
    """
    An `AdaptiveTuner` with good defaults for Sequence Classification tasks
    
    **Valid kwargs and defaults:**
      - `lr`:float = 0.001
      - `splitter`:function = `trainable_params`
      - `cbs`:list = None
      - `path`:Path = None
      - `model_dir`:Path = 'models'
      - `wd`:float = None
      - `wd_bn_bias`:bool = False
      - `train_bn`:bool = True
      - `moms`: tuple(float) = (0.95, 0.85, 0.95)
    
    """
    def __init__(
        self,
        dls:DataLoaders, # A set of DataLoaders
        model_name, # A HuggingFace model
        loss_func = CrossEntropyLossFlat(), # A loss function
        metrics = [accuracy, F1Score()], # Metrics to monitor the training with
        opt_func = Adam, # A fastai or torch Optimizer
        additional_cbs = None, # Additional Callbacks to have always tied to the Tuner,
        expose_fastai_api = False, # Whether to expose the fastai API
        **kwargs, # kwargs for `Learner.__init__`
    ):
        additional_cbs = listify(additional_cbs)
        for arg in 'dls,model,loss_func,metrics,opt_func,cbs,expose_fastai'.split(','): 
            if arg in kwargs.keys(): kwargs.pop(arg) # Pop all existing kwargs
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(dls[0].categorize.classes))
        
        super().__init__(
            expose_fastai_api,
            dls = dls, 
            model = model, 
            loss_func = loss_func, 
            metrics = metrics, 
            opt_func = opt_func, 
            cbs=additional_cbs, 
            **kwargs
        )
    
    @delegates(Learner.__init__)
    @classmethod
    def from_df(
        cls,
        df:pd.DataFrame, # A Pandas Dataframe or Path to a DataFrame
        text_col:str = 'text', # Name of the column the text is stored
        label_col:str = 'labels', # Name of the column the label(s) are stored
        model_name:str = None, # The string name of a huggingFace model
        split_func:callable = RandomSplitter(), # A function which splits the data
        loss_func = CrossEntropyLossFlat(), # A loss function
        metrics = [accuracy, F1Score()], # Metrics to monitor the training with
        batch_size=8, # A batch size
        collate_fn=default_data_collator, # An optional custom collate function
        opt_func = Adam, # A fastai or torch Optimizer
        additional_cbs = None, # Additional Callbacks to have always tied to the Tuner,
        expose_fastai_api = False, # Whether to expose the fastai API
        **kwargs # Learner kwargs
    ):
        "Convience method to build a `SequenceClassificationTuner` from a Pandas Dataframe"
        try:
            splits = split_func(df)
        except:
            splits = split_func(range_of(df))
        dls = SequenceClassificationDatasets.from_df(
            df,
            text_col,
            label_col,
            splits,
            tokenizer_name=model_name
        ).dataloaders(batch_size, collate_fn)
        
        return cls(dls, model_name, loss_func, metrics, opt_func, additional_cbs, expose_fastai_api)

In [ ]:
#export
from transformers import AutoModelForMaskedLM, AutoModelForCausalLM, AutoModelForSeq2SeqLM

In [ ]:
#export
mk_class('LMType', **{o:o.lower() for o in ['Masked', 'Causal', 'Seq2Seq']},
        doc="All valid language model classes with typo-proofing")

_constructors = {
    'masked':AutoModelForMaskedLM.from_pretrained,
    'causal':AutoModelForCausalLM.from_pretrained,
    'seq2seq':AutoModelForSeq2SeqLM.from_pretrained
                }

In [ ]:
#export
class LanguageModelTuner(AdaptiveTuner):
    """
    An `AdaptiveTuner` with good defaults for Language Model fine-tuning
    
    **Valid kwargs and defaults:**
      - `lr`:float = 0.001
      - `splitter`:function = `trainable_params`
      - `cbs`:list = None
      - `path`:Path = None
      - `model_dir`:Path = 'models'
      - `wd`:float = None
      - `wd_bn_bias`:bool = False
      - `train_bn`:bool = True
      - `moms`: tuple(float) = (0.95, 0.85, 0.95)
    
    """
    def __init__(
        self,
        dls:DataLoaders, # A set of DataLoaders
        model_name, # A HuggingFace model
        language_model_type:LMType = LMType.Causal, # The type of language model to use
        loss_func = CrossEntropyLossFlat(), # A loss function
        metrics = [accuracy, Perplexity()], # Metrics to monitor the training with
        opt_func = Adam, # A fastai or torch Optimizer
        additional_cbs = None, # Additional Callbacks to have always tied to the Tuner,
        expose_fastai_api = False, # Whether to expose the fastai API
        **kwargs, # kwargs for `Learner.__init__`
    ):
        additional_cbs = listify(additional_cbs)
        for arg in 'dls,model,loss_func,metrics,opt_func,cbs,expose_fastai'.split(','): 
            if arg in kwargs.keys(): kwargs.pop(arg) # Pop all existing kwargs
                
        if language_model_type is None: raise ValueError("Please specify the type of language model you want to use, such as `masked` or `causal`")
        if language_model_type not in _constructors.keys():
            raise ValueError(
                """
                Please enter a valid Langauge Model Type of:
                  * `masked` or `LMType.Masked`
                  * `causal` or `LMType.Causal`
                  * `seq2seq` or `LMType.Seq2Seq`
                """
            )
        try:
            model = _constructors[language_model_type](model_name)
        except Exception as e:
            message = e.args[0]
            m = f"Was not able to create a {language_model_type} instance of {model_name}. Please use a valid model for your task:"
            m += message
            e.args = [m]
            raise e
        
        super().__init__(
            expose_fastai_api,
            dls = dls, 
            model = model, 
            loss_func = loss_func, 
            metrics = metrics, 
            opt_func = opt_func, 
            cbs=additional_cbs, 
            **kwargs
        )
        
    @delegates(Learner.__init__)
    @classmethod
    def from_df(
        cls,
        df:pd.DataFrame, # A Pandas Dataframe or Path to a DataFrame
        text_col:str = 'text', # Name of the column the text is stored
        model_name:str = None, # The string name of a huggingFace model
        language_model_type:LMType = LMType.Causal, # The type of language model to use
        split_func:callable = RandomSplitter(), # A function which splits the data
        loss_func = CrossEntropyLossFlat(), # A loss function
        metrics = [accuracy, Perplexity()], # Metrics to monitor the training with
        batch_size=8, # A batch size
        collate_fn=default_collate, # An optional custom collate function
        opt_func = Adam, # A fastai or torch Optimizer
        additional_cbs = None, # Additional Callbacks to have always tied to the Tuner,
        expose_fastai_api = False, # Whether to expose the fastai API
        **kwargs # Learner kwargs
    ):
        "Convience method to build a `SequenceClassificationTuner` from a Pandas Dataframe"
        splits = split_func(range_of(df))
        dls = LanguageModelDatasets.from_df(
            df,
            text_col,
            splits,
            tokenizer_name=model_name
        ).dataloaders(batch_size, collate_fn)
        
        return cls(dls, model_name, language_model_type, loss_func, metrics, opt_func, additional_cbs, expose_fastai_api)

## Export - 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_file_utils.ipynb.
Converted 01_callback.ipynb.
Converted 02_model_hub.ipynb.
Converted 03_model.ipynb.
Converted 04_embeddings.ipynb.
Converted 04a_tutorial.embeddings.ipynb.
Converted 05_token_classification.ipynb.
Converted 05a_tutorial.token_tagging.ipynb.
Converted 06_sequence_classification.ipynb.
Converted 06a_tutorial.easy_sequence_classifier.ipynb.
Converted 07_summarization.ipynb.
Converted 07a_tutorial.summarization.ipynb.
Converted 08_translation.ipynb.
Converted 08a_tutorial.translation.ipynb.
Converted 09_text_generation.ipynb.
Converted 09a_tutorial.easy_text_generator.ipynb.
Converted 10_question_answering.ipynb.
Converted 10a_tutorial.question_answering.ipynb.
Converted 13a_transformers.squad_metrics.ipynb.
Converted 14_result.ipynb.
Converted 15_training.data.ipynb.
Converted 16_training.tuner.ipynb.
Converted 20_tutorial.tuner.sequence_classification.ipynb.
Converted 21_tutorial.tuner.language_model.ipynb.
Converted index.ipynb.
